In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from lifelines import CoxPHFitter

In [2]:
input_path = "E:\\deeplearning\\Hepatocarcinomes\\TCGA\\processed\\"
out_path = input_path + "heatmap\\survival\\"

In [3]:
data = pd.read_csv(input_path + 'fpkm_final_log2-median-centering.csv', sep='\t', index_col=0)
# data = pd.read_csv(input_path + "fpkm_final_log2-zscore_CCL3L1-noDEFB134.csv", sep='\t', header = None)
lsangro = pd.read_csv(input_path + 'Sangro_Table2_GeneList.csv', sep='\t', header = None)
os = pd.read_csv(input_path + 'Overall_Survival_legacy.txt', sep='\t')
dfs = pd.read_csv(input_path + 'Disease_Free_Survival_legacy.txt', sep='\t')

In [4]:
data = data.T
data.reset_index(inplace=True)
data = data.rename(columns={"index":"Sample"})
# data.columns = data.iloc[0]
# data = data.drop(data.index[0])
# data = data.rename(columns={"Gene Name":"Sample"})
# data = data.reset_index(drop=True)

In [5]:
print(data.shape) # (340, 734)
display(data.head(5))
print()
print(os.shape) # (340, 2)
display(os.head(5))
print()
print(dfs.shape) # (340, 2)
display(dfs.head(5))
print()

(340, 734)


Gene Name,Sample,A2M,ACVR1C,ADAM12,ADM,ADORA2A,AKT1,ALDOA,ALDOC,ANGPT1,...,WNT4,WNT5A,WNT5B,WNT7B,XCL1,ZAP70,ZC3H12A,ZEB1,ZEB2,CCL3L1
0,AADA-01A,0.580986,-0.042060,0.013547,-0.734360,0.716873,0.176478,-0.473462,-0.060915,0.120029,...,1.986813,-0.748394,0.791654,0.160138,0.135815,1.214148,-0.239418,0.170838,0.319786,0.819394
1,A5SJ-01A,0.979374,-0.071784,0.098334,0.301992,-0.105847,-0.428254,0.707962,0.268944,0.756201,...,0.911169,0.156321,0.257405,0.027693,0.167687,0.599456,2.378281,0.060643,0.026891,0.039873
2,AACF-01A,-0.436710,-0.055145,-0.007120,-0.439411,-0.342741,0.445181,-0.495030,-3.887819,-0.340829,...,1.964025,-0.724446,0.692565,-0.001063,0.598002,-0.032797,0.049167,-0.642371,-0.133674,0.670390
3,A9GZ-01A,2.521986,0.630880,-0.022410,-0.878293,0.411041,1.067124,-1.441670,0.723679,0.170189,...,2.395214,0.595501,2.513506,-0.013435,-0.185548,-0.227816,-0.164037,0.194186,-0.135175,-0.345550
4,A113-01A,0.771957,-0.012114,-0.000270,-1.283949,-0.609093,-0.882153,-0.216758,-0.961504,-0.117368,...,0.981836,-0.222799,0.806006,0.011926,0.641115,-0.244449,-0.531706,-0.187144,-0.079972,-0.050226



(440, 4)


,Study ID,Patient ID,OS_STATUS,OS_MONTHS
0,lihc_tcga,TCGA-2V-A95S,0:LIVING,NaN
1,lihc_tcga,TCGA-2Y-A9GS,1:DECEASED,23.78
2,lihc_tcga,TCGA-2Y-A9GT,1:DECEASED,53.35
3,lihc_tcga,TCGA-2Y-A9GU,0:LIVING,63.70
4,lihc_tcga,TCGA-2Y-A9GV,1:DECEASED,83.18



(440, 4)


,Study ID,Patient ID,DFS_STATUS,DFS_MONTHS
0,lihc_tcga,TCGA-2V-A95S,0:DiseaseFree,NaN
1,lihc_tcga,TCGA-2Y-A9GS,1:Recurred/Progressed,3.35
2,lihc_tcga,TCGA-2Y-A9GT,1:Recurred/Progressed,35.58
3,lihc_tcga,TCGA-2Y-A9GU,0:DiseaseFree,63.70
4,lihc_tcga,TCGA-2Y-A9GV,1:Recurred/Progressed,71.91


### Subset to keep only genes from Sangro's list

In [6]:
lsangro = pd.read_csv(input_path + 'Sangro_Table2_GeneList.csv', sep='\t', header=None)
lsangro.rename(columns={0:"Gene Name"}, inplace=True)
print(lsangro.shape)
display(lsangro.head(5))

(29, 1)


,Gene Name
0,CD274(PDL1)
1,CD8A
2,LAG3
3,STAT1
4,CCL2


In [7]:
lsangro.iloc[0,0] = "CD274" # 'CD274(PDL1)'
lsangro.drop(lsangro[lsangro["Gene Name"]=="CCL3"].index, inplace=True)
print(lsangro.shape)
display(lsangro.head(5))

(28, 1)


,Gene Name
0,CD274
1,CD8A
2,LAG3
3,STAT1
4,CCL2


In [8]:
lsangro = lsangro["Gene Name"].values.tolist()
lsangro.insert(0, "Sample")
data = data[lsangro]
print(data.shape) # (340, 29)
display(data.head(5))

(340, 29)


Gene Name,Sample,CD274,CD8A,LAG3,STAT1,CCL2,CCL4,CXCL10,CXCL9,GZMK,...,CCL5,CD27,CXCR6,CD276,PDCD1LG2,TIGIT,CCR5,CXCL11,GZMA,PRF1
0,AADA-01A,-0.024299,1.220583,1.349095,-0.338349,1.431152,0.511898,-1.014153,0.916793,2.113736,...,1.904710,1.187266,0.555145,-0.272298,0.063651,0.258454,0.796701,-0.189377,1.844013,0.856193
1,A5SJ-01A,0.162838,1.140263,0.707823,0.428330,0.498463,0.590076,1.611416,3.295769,1.118066,...,1.409919,1.645123,0.422194,-0.419198,0.266429,0.628489,0.699625,1.558460,0.630920,0.088949
2,AACF-01A,0.105091,-0.026725,-0.028561,1.457781,-0.750145,0.000107,1.480531,-0.595409,0.113313,...,0.010616,-0.480303,0.036483,-0.062031,0.262537,-0.040435,-0.068163,1.191091,0.072777,0.474411
3,A9GZ-01A,0.032817,-0.407066,-0.006508,-1.050598,-0.823095,-0.401642,-2.233747,-1.314490,-0.680796,...,-1.530659,0.025447,-0.204613,-0.273227,-0.225024,-0.036404,-0.254229,-0.459448,-0.678391,-0.068765
4,A113-01A,-0.026098,0.414796,-0.223885,0.615622,-1.387357,-0.016011,-0.883621,0.769023,-0.367766,...,-0.292722,0.459020,0.007975,-0.471770,-0.030553,0.159172,0.014350,-0.002096,-0.582563,-0.580989


### Extract survival data

In [9]:
sample = data["Sample"][0]

In [10]:
# Here is no chained operation. SettingWithCopyWarning maybe be caused by created item column, try to build the df again.
def exact_info(df, item):
    i=0
    data[item] = ''
    for sample in data["Sample"]:
        info = set(df[df['Patient ID'].str.contains(sample[:-4][-4:])][item])
        if len(info) != 1 :
#             raise ValueError('Multiple or no values found.')
            i=i+1
        else:
            info = list(info)[0]
            data.loc[data["Sample"]==sample, item] = info
    return i # count of samples not found

In [11]:
print(exact_info(df = os, item = 'OS_STATUS')) # 0
print(exact_info(df = os, item = 'OS_MONTHS')) # 0

0
0


In [12]:
print(exact_info(df = dfs, item = 'DFS_STATUS')) # 0
print(exact_info(df = dfs, item = 'DFS_MONTHS')) # 0

0
0


In [13]:
print(data.shape) # (340, 33)
display(data.head(5))
print()

(340, 33)


Gene Name,Sample,CD274,CD8A,LAG3,STAT1,CCL2,CCL4,CXCL10,CXCL9,GZMK,...,PDCD1LG2,TIGIT,CCR5,CXCL11,GZMA,PRF1,OS_STATUS,OS_MONTHS,DFS_STATUS,DFS_MONTHS
0,AADA-01A,-0.024299,1.220583,1.349095,-0.338349,1.431152,0.511898,-1.014153,0.916793,2.113736,...,0.063651,0.258454,0.796701,-0.189377,1.844013,0.856193,0:LIVING,40.51,0:DiseaseFree,40.51
1,A5SJ-01A,0.162838,1.140263,0.707823,0.428330,0.498463,0.590076,1.611416,3.295769,1.118066,...,0.266429,0.628489,0.699625,1.558460,0.630920,0.088949,0:LIVING,22.93,1:Recurred/Progressed,10.12
2,AACF-01A,0.105091,-0.026725,-0.028561,1.457781,-0.750145,0.000107,1.480531,-0.595409,0.113313,...,0.262537,-0.040435,-0.068163,1.191091,0.072777,0.474411,1:DECEASED,11.99,1:Recurred/Progressed,3.91
3,A9GZ-01A,0.032817,-0.407066,-0.006508,-1.050598,-0.823095,-0.401642,-2.233747,-1.314490,-0.680796,...,-0.225024,-0.036404,-0.254229,-0.459448,-0.678391,-0.068765,1:DECEASED,27.86,1:Recurred/Progressed,23.62
4,A113-01A,-0.026098,0.414796,-0.223885,0.615622,-1.387357,-0.016011,-0.883621,0.769023,-0.367766,...,-0.030553,0.159172,0.014350,-0.002096,-0.582563,-0.580989,0:LIVING,79.66,1:Recurred/Progressed,55.06


### Preprocess

In [14]:
for i in range(len(data)):
    data.loc[i,'OS_STATUS'] = data.loc[i,'OS_STATUS'][0]
    try:
        data.loc[i,'DFS_STATUS'] = data.loc[i,'DFS_STATUS'][0]
    except:
        pass

In [15]:
display(data.dtypes[-5:])
data = data.astype({'OS_STATUS':'float64', 'OS_MONTHS':'float64', 'DFS_STATUS':'float64', 'DFS_MONTHS':'float64'})
display(data.dtypes[-5:])

Gene Name
PRF1          float64
OS_STATUS      object
OS_MONTHS      object
DFS_STATUS     object
DFS_MONTHS     object
dtype: object

Gene Name
PRF1          float64
OS_STATUS     float64
OS_MONTHS     float64
DFS_STATUS    float64
DFS_MONTHS    float64
dtype: object

In [16]:
print(data.shape) # (340, 33)
display(data.head(5))
print()

(340, 33)


Gene Name,Sample,CD274,CD8A,LAG3,STAT1,CCL2,CCL4,CXCL10,CXCL9,GZMK,...,PDCD1LG2,TIGIT,CCR5,CXCL11,GZMA,PRF1,OS_STATUS,OS_MONTHS,DFS_STATUS,DFS_MONTHS
0,AADA-01A,-0.024299,1.220583,1.349095,-0.338349,1.431152,0.511898,-1.014153,0.916793,2.113736,...,0.063651,0.258454,0.796701,-0.189377,1.844013,0.856193,0.0,40.51,0.0,40.51
1,A5SJ-01A,0.162838,1.140263,0.707823,0.428330,0.498463,0.590076,1.611416,3.295769,1.118066,...,0.266429,0.628489,0.699625,1.558460,0.630920,0.088949,0.0,22.93,1.0,10.12
2,AACF-01A,0.105091,-0.026725,-0.028561,1.457781,-0.750145,0.000107,1.480531,-0.595409,0.113313,...,0.262537,-0.040435,-0.068163,1.191091,0.072777,0.474411,1.0,11.99,1.0,3.91
3,A9GZ-01A,0.032817,-0.407066,-0.006508,-1.050598,-0.823095,-0.401642,-2.233747,-1.314490,-0.680796,...,-0.225024,-0.036404,-0.254229,-0.459448,-0.678391,-0.068765,1.0,27.86,1.0,23.62
4,A113-01A,-0.026098,0.414796,-0.223885,0.615622,-1.387357,-0.016011,-0.883621,0.769023,-0.367766,...,-0.030553,0.159172,0.014350,-0.002096,-0.582563,-0.580989,0.0,79.66,1.0,55.06


In [17]:
# # Export data
# data.to_csv(input_path+"data_median_survival.csv",sep="\t", index=False)

### Explore data

In [18]:
# Only 1 missing data for OS, with OS_STATUS but no OS_MONTHS
data[data['OS_MONTHS'].isna()]

Gene Name,Sample,CD274,CD8A,LAG3,STAT1,CCL2,CCL4,CXCL10,CXCL9,GZMK,...,PDCD1LG2,TIGIT,CCR5,CXCL11,GZMA,PRF1,OS_STATUS,OS_MONTHS,DFS_STATUS,DFS_MONTHS
33,A95S-01A,0.46826,3.238845,2.840833,4.188422,0.414246,1.822712,5.929623,4.996384,2.401558,...,0.894573,1.623585,1.848018,4.139949,3.163895,2.876739,0.0,NaN,0.0,NaN


In [19]:
# Censored values
data.groupby(["OS_STATUS"]).count()

Gene Name,Sample,CD274,CD8A,LAG3,STAT1,CCL2,CCL4,CXCL10,CXCL9,GZMK,...,CD276,PDCD1LG2,TIGIT,CCR5,CXCL11,GZMA,PRF1,OS_MONTHS,DFS_STATUS,DFS_MONTHS
OS_STATUS,,,,,,,,,,,,,,,,,,,,,
0.0,220,220,220,220,220,220,220,220,220,220,...,220,220,220,220,220,220,220,219,220,219
1.0,120,120,120,120,120,120,120,120,120,120,...,120,120,120,120,120,120,120,120,77,77


## Overall survival

### Visualize censored values

In [20]:
# ax = plot_lifetimes(T, event_observed=E)
# # ax.set_xlim(0, 25)
# # ax.vlines(10, 0, 30, lw=2, linestyles='--')
# ax.set_xlabel("time")
# ax.set_title("Overall survival time")
# # print("Observed lifetimes at time %d:\n" % (CURRENT_TIME), observed_lifetimes)

### Cox proportional hazards regression model

#### Cox with only Sangro's list

In [21]:
data_dummy = data.dropna(subset = ["OS_MONTHS"]).reset_index(drop=True).drop(columns=["Sample", "DFS_STATUS", "DFS_MONTHS"])
print(data_dummy.shape) # (339, 30)
data_dummy.head(5)

(339, 30)


Gene Name,CD274,CD8A,LAG3,STAT1,CCL2,CCL4,CXCL10,CXCL9,GZMK,HLA-DMA,...,CXCR6,CD276,PDCD1LG2,TIGIT,CCR5,CXCL11,GZMA,PRF1,OS_STATUS,OS_MONTHS
0,-0.024299,1.220583,1.349095,-0.338349,1.431152,0.511898,-1.014153,0.916793,2.113736,-0.455090,...,0.555145,-0.272298,0.063651,0.258454,0.796701,-0.189377,1.844013,0.856193,0.0,40.51
1,0.162838,1.140263,0.707823,0.428330,0.498463,0.590076,1.611416,3.295769,1.118066,0.351409,...,0.422194,-0.419198,0.266429,0.628489,0.699625,1.558460,0.630920,0.088949,0.0,22.93
2,0.105091,-0.026725,-0.028561,1.457781,-0.750145,0.000107,1.480531,-0.595409,0.113313,0.000915,...,0.036483,-0.062031,0.262537,-0.040435,-0.068163,1.191091,0.072777,0.474411,1.0,11.99
3,0.032817,-0.407066,-0.006508,-1.050598,-0.823095,-0.401642,-2.233747,-1.314490,-0.680796,-1.604050,...,-0.204613,-0.273227,-0.225024,-0.036404,-0.254229,-0.459448,-0.678391,-0.068765,1.0,27.86
4,-0.026098,0.414796,-0.223885,0.615622,-1.387357,-0.016011,-0.883621,0.769023,-0.367766,-0.984838,...,0.007975,-0.471770,-0.030553,0.159172,0.014350,-0.002096,-0.582563,-0.580989,0.0,79.66


In [22]:
# Using Cox Proportional Hazards model without on hot encoding
cph = CoxPHFitter()   ## Instantiate the class to create a cph object
print(cph.fit(data_dummy, "OS_MONTHS", "OS_STATUS"))   ## Fit the data to train the model
cph.print_summary()    ## Have a look at the significance of the features

<lifelines.CoxPHFitter: fitted with 339 total observations, 219 right-censored observations>


In [23]:
cph.summary[cph.summary["p"]<0.1]

,coef,exp(coef),se(coef),coef lower 95%,coef upper 95%,exp(coef) lower 95%,exp(coef) upper 95%,z,p,-log2(p)
Gene Name,,,,,,,,,,
CCL4,0.414234,1.513212,0.218483,-0.013984,0.842453,0.986113,2.322056,1.895958,0.057966,4.108659
HLA-DMB,0.715431,2.045068,0.411626,-0.091341,1.522203,0.912706,4.582311,1.738060,0.082200,3.604715
IDO1,0.284146,1.328627,0.158732,-0.026963,0.595254,0.973398,1.813492,1.790100,0.073438,3.767334


In [24]:
# Inflammatory signature
cph = CoxPHFitter()   ## Instantiate the class to create a cph object
print(cph.fit(data_dummy[["CD274","CD8A","LAG3","STAT1","OS_MONTHS", "OS_STATUS"]], "OS_MONTHS", "OS_STATUS"))   ## Fit the data to train the model
cph.summary[cph.summary["p"]<0.1]    ## Have a look at the significance of the features

<lifelines.CoxPHFitter: fitted with 339 total observations, 219 right-censored observations>


,coef,exp(coef),se(coef),coef lower 95%,coef upper 95%,exp(coef) lower 95%,exp(coef) upper 95%,z,p,-log2(p)
Gene Name,,,,,,,,,,
CD8A,-0.456604,0.633431,0.190195,-0.829378,-0.083829,0.436321,0.919588,-2.400720,0.016363,5.933431
LAG3,0.263578,1.301578,0.155364,-0.040929,0.568085,0.959897,1.764884,1.696521,0.089787,3.477345


In [25]:
# Gajewski 13-Gene Inflammatory signature
cph = CoxPHFitter()   ## Instantiate the class to create a cph object
print(cph.fit(data_dummy[["CCL2","CCL4","CD8A","CXCL10","CXCL9","GZMK","HLA-DMA","HLA-DMB","HLA-DOA","HLA-DOB","ICOS","IRF1",
                          "OS_MONTHS", "OS_STATUS"]], "OS_MONTHS", "OS_STATUS"))   ## Fit the data to train the model
cph.summary[cph.summary["p"]<0.1]    ## Have a look at the significance of the features

<lifelines.CoxPHFitter: fitted with 339 total observations, 219 right-censored observations>


,coef,exp(coef),se(coef),coef lower 95%,coef upper 95%,exp(coef) lower 95%,exp(coef) upper 95%,z,p,-log2(p)
Gene Name,,,,,,,,,,
CCL4,0.438657,1.550623,0.192063,0.062220,0.815094,1.064196,2.259388,2.283920,0.022376,5.481888
CD8A,-0.570713,0.565122,0.244024,-1.048992,-0.092435,0.350291,0.911708,-2.338760,0.019348,5.691682
ICOS,1.415816,4.119847,0.598169,0.243426,2.588207,1.275611,13.305887,2.366915,0.017937,5.800914


In [26]:
# 6-Gene Interferon Gamma signature
cph = CoxPHFitter()   ## Instantiate the class to create a cph object
print(cph.fit(data_dummy[["CXCL10","CXCL9","HLA-DRA","IDO1","IFNG","STAT1","OS_MONTHS", "OS_STATUS"]], "OS_MONTHS", "OS_STATUS"))   ## Fit the data to train the model
cph.summary[cph.summary["p"]<0.1]    ## Have a look at the significance of the features

<lifelines.CoxPHFitter: fitted with 339 total observations, 219 right-censored observations>


,coef,exp(coef),se(coef),coef lower 95%,coef upper 95%,exp(coef) lower 95%,exp(coef) upper 95%,z,p,-log2(p)
Gene Name,,,,,,,,,,
HLA-DRA,-0.165258,0.847675,0.088427,-0.338572,0.008057,0.712787,1.008089,-1.868851,0.061643,4.019908
IDO1,0.244756,1.277310,0.135158,-0.020149,0.509662,0.980053,1.664728,1.810890,0.070158,3.833252


In [27]:
# Interferon Gamma Biology signature
cph = CoxPHFitter()   ## Instantiate the class to create a cph object
print(cph.fit(data_dummy[["CCL5","CD27","CXCL9","CXCR6","IDO1","STAT1","OS_MONTHS", "OS_STATUS"]], "OS_MONTHS", "OS_STATUS"))   ## Fit the data to train the model
cph.summary[cph.summary["p"]<0.1]   ## Have a look at the significance of the features

<lifelines.CoxPHFitter: fitted with 339 total observations, 219 right-censored observations>


,coef,exp(coef),se(coef),coef lower 95%,coef upper 95%,exp(coef) lower 95%,exp(coef) upper 95%,z,p,-log2(p)
Gene Name,,,,,,,,,,
IDO1,0.290144,1.33662,0.137635,0.020384,0.559904,1.020594,1.750504,2.108068,0.035025,4.835468


In [28]:
# T-cell Exhaustion signature
cph = CoxPHFitter()   ## Instantiate the class to create a cph object
print(cph.fit(data_dummy[["CD274","CD276","CD8A","LAG3","PDCD1LG2","TIGIT","OS_MONTHS", "OS_STATUS"]], "OS_MONTHS", "OS_STATUS"))   ## Fit the data to train the model
cph.summary[cph.summary["p"]<0.1]    ## Have a look at the significance of the features

<lifelines.CoxPHFitter: fitted with 339 total observations, 219 right-censored observations>


,coef,exp(coef),se(coef),coef lower 95%,coef upper 95%,exp(coef) lower 95%,exp(coef) upper 95%,z,p,-log2(p)
Gene Name,,,,,,,,,,
CD8A,-0.891501,0.410040,0.266528,-1.413886,-0.369116,0.243196,0.691345,-3.344870,0.000823,10.246451
TIGIT,1.204917,3.336482,0.652829,-0.074605,2.484439,0.928110,11.994389,1.845684,0.064938,3.944792


In [29]:
# Ribas 10-Gene Interferon Gamma signature
cph = CoxPHFitter()   ## Instantiate the class to create a cph object
print(cph.fit(data_dummy[["CCR5","CXCL10","CXCL11","CXCL9","GZMA","HLA-DRA","IDO1","IFNG","PRF1","STAT1","OS_MONTHS", "OS_STATUS"]], "OS_MONTHS", "OS_STATUS"))   ## Fit the data to train the model
cph.summary[cph.summary["p"]<0.1]    ## Have a look at the significance of the features

<lifelines.CoxPHFitter: fitted with 339 total observations, 219 right-censored observations>


,coef,exp(coef),se(coef),coef lower 95%,coef upper 95%,exp(coef) lower 95%,exp(coef) upper 95%,z,p,-log2(p)
Gene Name,,,,,,,,,,
IDO1,0.315249,1.370600,0.142134,0.036672,0.593826,1.037353,1.810904,2.217974,0.026557,5.234785
IFNG,0.756150,2.130059,0.387338,-0.003019,1.515318,0.996986,4.550869,1.952170,0.050918,4.295680


In [30]:
# cph.summary[cph.summary["coef"]>2]

In [31]:
# # Check if the proprtional hazards assuption is violated
# cph.check_assumptions(data_dummy, show_plots=True)

## Disease free survival

### Cox proportional hazards regression model

#### Cox with only Sangro's list

In [32]:
data_dummy = data.dropna(subset = ["DFS_MONTHS"]).reset_index(drop=True).drop(columns=["Sample", "OS_STATUS", "OS_MONTHS"])
print(data_dummy.shape) # (296, 30)
data_dummy.head(5)

(296, 30)


Gene Name,CD274,CD8A,LAG3,STAT1,CCL2,CCL4,CXCL10,CXCL9,GZMK,HLA-DMA,...,CXCR6,CD276,PDCD1LG2,TIGIT,CCR5,CXCL11,GZMA,PRF1,DFS_STATUS,DFS_MONTHS
0,-0.024299,1.220583,1.349095,-0.338349,1.431152,0.511898,-1.014153,0.916793,2.113736,-0.455090,...,0.555145,-0.272298,0.063651,0.258454,0.796701,-0.189377,1.844013,0.856193,0.0,40.51
1,0.162838,1.140263,0.707823,0.428330,0.498463,0.590076,1.611416,3.295769,1.118066,0.351409,...,0.422194,-0.419198,0.266429,0.628489,0.699625,1.558460,0.630920,0.088949,1.0,10.12
2,0.105091,-0.026725,-0.028561,1.457781,-0.750145,0.000107,1.480531,-0.595409,0.113313,0.000915,...,0.036483,-0.062031,0.262537,-0.040435,-0.068163,1.191091,0.072777,0.474411,1.0,3.91
3,0.032817,-0.407066,-0.006508,-1.050598,-0.823095,-0.401642,-2.233747,-1.314490,-0.680796,-1.604050,...,-0.204613,-0.273227,-0.225024,-0.036404,-0.254229,-0.459448,-0.678391,-0.068765,1.0,23.62
4,-0.026098,0.414796,-0.223885,0.615622,-1.387357,-0.016011,-0.883621,0.769023,-0.367766,-0.984838,...,0.007975,-0.471770,-0.030553,0.159172,0.014350,-0.002096,-0.582563,-0.580989,1.0,55.06


In [33]:
# Using Cox Proportional Hazards model without on hot encoding
cph = CoxPHFitter()   ## Instantiate the class to create a cph object
print(cph.fit(data_dummy, "DFS_MONTHS", "DFS_STATUS"))   ## Fit the data to train the model
cph.print_summary()    ## Have a look at the significance of the features

<lifelines.CoxPHFitter: fitted with 296 total observations, 133 right-censored observations>


In [34]:
cph.summary[cph.summary["p"]<0.1]

,coef,exp(coef),se(coef),coef lower 95%,coef upper 95%,exp(coef) lower 95%,exp(coef) upper 95%,z,p,-log2(p)
Gene Name,,,,,,,,,,
CCL4,0.348203,1.416519,0.182195,-0.008894,0.705299,0.991146,2.024452,1.911150,0.055985,4.158808
HLA-DRA,-0.636050,0.529379,0.204132,-1.036142,-0.235958,0.354821,0.789814,-3.115871,0.001834,9.090772
IDO1,0.291053,1.337836,0.145906,0.005083,0.577024,1.005096,1.780731,1.994801,0.046065,4.440199
PRF1,-0.894495,0.408814,0.305558,-1.493377,-0.295613,0.224613,0.744075,-2.927418,0.003418,8.192676


In [35]:
# Inflammatory signature
cph = CoxPHFitter()   ## Instantiate the class to create a cph object
print(cph.fit(data_dummy[["CD274","CD8A","LAG3","STAT1","DFS_MONTHS", "DFS_STATUS"]], "DFS_MONTHS", "DFS_STATUS"))   ## Fit the data to train the model
cph.summary[cph.summary["p"]<0.1]    ## Have a look at the significance of the features

<lifelines.CoxPHFitter: fitted with 296 total observations, 133 right-censored observations>


,coef,exp(coef),se(coef),coef lower 95%,coef upper 95%,exp(coef) lower 95%,exp(coef) upper 95%,z,p,-log2(p)
Gene Name,,,,,,,,,,
CD8A,-0.465431,0.627865,0.167455,-0.793636,-0.137226,0.452198,0.871773,-2.779446,0.005445,7.520808
LAG3,0.295338,1.343580,0.150154,0.001040,0.589635,1.001041,1.803330,1.966893,0.049195,4.345330


In [36]:
# Gajewski 13-Gene Inflammatory signature
cph = CoxPHFitter()   ## Instantiate the class to create a cph object
print(cph.fit(data_dummy[["CCL2","CCL4","CD8A","CXCL10","CXCL9","GZMK","HLA-DMA","HLA-DMB","HLA-DOA","HLA-DOB","ICOS","IRF1",
                          "DFS_MONTHS", "DFS_STATUS"]], "DFS_MONTHS", "DFS_STATUS"))   ## Fit the data to train the model
cph.summary[cph.summary["p"]<0.1]    ## Have a look at the significance of the features

<lifelines.CoxPHFitter: fitted with 296 total observations, 133 right-censored observations>


,coef,exp(coef),se(coef),coef lower 95%,coef upper 95%,exp(coef) lower 95%,exp(coef) upper 95%,z,p,-log2(p)
Gene Name,,,,,,,,,,
CD8A,-0.624330,0.535620,0.205485,-1.027073,-0.221588,0.358054,0.801245,-3.038333,0.002379,8.715483
ICOS,1.294881,3.650562,0.520086,0.275532,2.314231,1.317231,10.117136,2.489745,0.012783,6.289577


In [37]:
# 6-Gene Interferon Gamma signature
cph = CoxPHFitter()   ## Instantiate the class to create a cph object
print(cph.fit(data_dummy[["CXCL10","CXCL9","HLA-DRA","IDO1","IFNG","STAT1","DFS_MONTHS", "DFS_STATUS"]], "DFS_MONTHS", "DFS_STATUS"))   ## Fit the data to train the model
cph.summary[cph.summary["p"]<0.1]    ## Have a look at the significance of the features

<lifelines.CoxPHFitter: fitted with 296 total observations, 133 right-censored observations>


,coef,exp(coef),se(coef),coef lower 95%,coef upper 95%,exp(coef) lower 95%,exp(coef) upper 95%,z,p,-log2(p)
Gene Name,,,,,,,,,,
HLA-DRA,-0.296975,0.743062,0.075787,-0.445516,-0.148435,0.640494,0.862056,-3.918531,0.000089,13.454374
IDO1,0.315205,1.370540,0.129301,0.061779,0.568630,1.063728,1.765847,2.437759,0.014779,6.080343


In [38]:
# Interferon Gamma Biology signature
cph = CoxPHFitter()   ## Instantiate the class to create a cph object
print(cph.fit(data_dummy[["CCL5","CD27","CXCL9","CXCR6","IDO1","STAT1","DFS_MONTHS", "DFS_STATUS"]], "DFS_MONTHS", "DFS_STATUS"))   ## Fit the data to train the model
cph.summary[cph.summary["p"]<0.1]    ## Have a look at the significance of the features

<lifelines.CoxPHFitter: fitted with 296 total observations, 133 right-censored observations>


,coef,exp(coef),se(coef),coef lower 95%,coef upper 95%,exp(coef) lower 95%,exp(coef) upper 95%,z,p,-log2(p)
Gene Name,,,,,,,,,,
IDO1,0.222626,1.249354,0.131723,-0.035545,0.480798,0.965079,1.617364,1.690115,0.091006,3.457896


In [39]:
# T-cell Exhaustion signature
cph = CoxPHFitter()   ## Instantiate the class to create a cph object
print(cph.fit(data_dummy[["CD274","CD276","CD8A","LAG3","PDCD1LG2","TIGIT","DFS_MONTHS", "DFS_STATUS"]], "DFS_MONTHS", "DFS_STATUS"))   ## Fit the data to train the model
cph.summary[cph.summary["p"]<0.1]    ## Have a look at the significance of the features

<lifelines.CoxPHFitter: fitted with 296 total observations, 133 right-censored observations>


,coef,exp(coef),se(coef),coef lower 95%,coef upper 95%,exp(coef) lower 95%,exp(coef) upper 95%,z,p,-log2(p)
Gene Name,,,,,,,,,,
CD8A,-0.719758,0.486870,0.213491,-1.138192,-0.301323,0.320398,0.739839,-3.371374,0.000748,10.384784
TIGIT,0.947271,2.578664,0.567780,-0.165558,2.060100,0.847421,7.846757,1.668376,0.095241,3.392273


In [40]:
# Ribas 10-Gene Interferon Gamma signature
cph = CoxPHFitter()   ## Instantiate the class to create a cph object
print(cph.fit(data_dummy[["CCR5","CXCL10","CXCL11","CXCL9","GZMA","HLA-DRA","IDO1","IFNG","PRF1","STAT1","DFS_MONTHS", 
                          "DFS_STATUS"]], "DFS_MONTHS", "DFS_STATUS"))   ## Fit the data to train the model
cph.summary[cph.summary["p"]<0.1]    ## Have a look at the significance of the features

<lifelines.CoxPHFitter: fitted with 296 total observations, 133 right-censored observations>


,coef,exp(coef),se(coef),coef lower 95%,coef upper 95%,exp(coef) lower 95%,exp(coef) upper 95%,z,p,-log2(p)
Gene Name,,,,,,,,,,
GZMA,0.423991,1.528048,0.179018,0.073123,0.774860,1.075863,2.170287,2.368430,0.017864,5.806821
HLA-DRA,-0.276501,0.758433,0.099531,-0.471578,-0.081424,0.624017,0.921803,-2.778040,0.005469,7.514564
IDO1,0.396961,1.487297,0.137557,0.127354,0.666568,1.135819,1.947541,2.885789,0.003904,8.000705
PRF1,-0.854065,0.425681,0.241451,-1.327300,-0.380831,0.265192,0.683293,-3.537226,0.000404,11.272094


In [41]:
# # Check if the proprtional hazards assuption is violated
# cph.check_assumptions(data_dummy, show_plots=True)

In [42]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [43]:
# Compute and view VIF
vif = pd.DataFrame()
vif["variables"] = data_dummy.iloc[:,:-2].columns
vif["VIF"] = [variance_inflation_factor(data_dummy.iloc[:,:-2].values, i) for i in range(data_dummy.iloc[:,:-2].shape[1])]

# View results using print
print(vif)

   variables        VIF
0      CD274   1.977624
1       CD8A  16.282401
2       LAG3   7.739724
3      STAT1   2.320549
4       CCL2   2.360463
5       CCL4   5.834941
6     CXCL10   4.299322
7      CXCL9   5.346448
8       GZMK  11.788467
9    HLA-DMA   8.943657
10   HLA-DMB  22.034881
11   HLA-DOA   8.496769
12   HLA-DOB   3.590300
13      ICOS  14.293404
14      IRF1   3.423938
15   HLA-DRA  14.291868
16      IDO1   2.326094
17      IFNG   6.753647
18      CCL5  17.330324
19      CD27  18.374891
20     CXCR6  11.621912
21     CD276   1.528188
22  PDCD1LG2   4.333860
23     TIGIT  25.905036
24      CCR5  12.265705
25    CXCL11   3.395500
26      GZMA  22.312132
27      PRF1  10.667189


In [44]:
str(vif["variables"].values.tolist()).replace("', '","+").replace("-",".")

"['CD274+CD8A+LAG3+STAT1+CCL2+CCL4+CXCL10+CXCL9+GZMK+HLA.DMA+HLA.DMB+HLA.DOA+HLA.DOB+ICOS+IRF1+HLA.DRA+IDO1+IFNG+CCL5+CD27+CXCR6+CD276+PDCD1LG2+TIGIT+CCR5+CXCL11+GZMA+PRF1']"